We read model:

In [25]:
# First, we can import some functions so we can use the model
from cobra.io import read_sbml_model, write_sbml_model
from cobra import Reaction, Metabolite

# Second, we can read the GEM and save it as ‘model’
model = read_sbml_model('data/iML1515.xml')

# Third, we can show general information about the loaded model
model

Name,iML1515
Memory address,1ec5de2c410
Number of metabolites,1877
Number of reactions,2712
Number of genes,1516
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


We add reactions and metabolites:

In [26]:
for metabolite in model.metabolites.query('Acetate', 'name'):
    print(metabolite.name, metabolite.id)                           # ac_c

for metabolite in model.metabolites.query('Acetyl-CoA', 'name'):
    print(metabolite.name, metabolite.id)                           # accoa_c
    
for metabolite in model.metabolites.query('Acetoacetyl-CoA', 'name'):
    print(metabolite.name, metabolite.id)                           # aacoa_c
    
for metabolite in model.metabolites.query('3-hydroxybutyryl-CoA', 'name'):
    print(metabolite.name, metabolite.id)                           # NA
    
for reaction in model.reactions.query('Acetyl-CoA', 'name'):
    print(reaction.name, reaction)                                  # Reaction for phaA corresponds to ACACT1r
    
for reaction in model.reactions.query('Acetate', 'name'):
    print(reaction.name, reaction)                                  # Reaction for acs ACS

Acetate ac_c
Acetate ac_e
Acetate ac_p
Acetyl-CoA accoa_c
Acetoacetyl-CoA aacoa_c
Acetyl-CoA synthetase ACS: ac_c + atp_c + coa_c --> accoa_c + amp_c + ppi_c
Acetyl-CoA carboxylase ACCOAC: accoa_c + atp_c + hco3_c --> adp_c + h_c + malcoa_c + pi_c
Acetyl-CoA:acetoacetyl-CoA transferase ACACCT: acac_c + accoa_c --> aacoa_c + ac_c
Acetyl-CoA:butyrate-CoA transferase BUTCT: accoa_c + but_c --> ac_c + btcoa_c
Acetyl-CoA ACP transacylase ACOATA: ACP_c + accoa_c <=> acACP_c + coa_c
Acetyl-CoA C-acyltransferase (tetradecanoyl-CoA) (r) ACACT7r: accoa_c + tdcoa_c <=> 3ohdcoa_c + coa_c
Acetyl-CoA C-acyltransferase (butanoyl-CoA) (r) ACACT2r: accoa_c + btcoa_c <=> 3ohcoa_c + coa_c
Acetyl-CoA C-acyltransferase (hexanoyl-CoA) (r) ACACT3r: accoa_c + hxcoa_c <=> 3oocoa_c + coa_c
Acetyl-CoA C-acyltransferase (octanoyl-CoA) (r) ACACT4r: accoa_c + occoa_c <=> 3odcoa_c + coa_c
Acetyl-CoA C-acyltransferase (decanoyl-CoA) (r) ACACT5r: accoa_c + dcacoa_c <=> 3oddcoa_c + coa_c
Acetyl-CoA C-acyltransferase (d

In [27]:
# phaB reaction

new_reaction1 = Reaction('phaB', 'Acetoacetyl-CoA: 3-Hydroxybutyryl-CoA reductase')  # The reaction corresponding to phaB

# phaB uses:
## Acetoacetyl-CoA + NADPH + H+ <=> 3-Hydroxybutyryl-CoA + NADP+
## We can find these as: (Note: All of these are in the cytoplasm)
### Acetoacetyl-CoA      = aacoa_c
### NADPH                = nadph_c
### H+                   = h_c
### 
### 3-Hydroxybutyryl-CoA = Not yet defined
### NADP+                = nadp_c

# Since 3-Hydroxybutyryl-CoA does not exist in the model, we will have to define it before.
hydroxybutyrylCoA = Metabolite(id='hbtcoa_c', name='3-Hydroxybutyryl-CoA', compartment='c', formula='C25H42N7O18P3S')


# We can now define the reaction stoichiometry:
new_reaction1.add_metabolites({
                            model.metabolites.get_by_id("aacoa_c"): -1,
                            model.metabolites.get_by_id("nadph_c"): -1,
                            model.metabolites.get_by_id("h_c"): -1,
                            hydroxybutyrylCoA: 1,
                            model.metabolites.get_by_id("nadp_c"): 1
                             })

model.add_reactions([new_reaction1])

In [28]:
model.reactions.phaB

Reaction identifier,phaB
Name,Acetoacetyl-CoA: 3-Hydroxybutyryl-CoA reductase
Memory address,0x1ec5d7d4210
Stoichiometry,aacoa_c + h_c + nadph_c --> hbtcoa_c + nadp_c Acetoacetyl-CoA + H+ + Nicotinamide adenine dinucleotide phosphate - reduced --> 3-Hydroxybutyryl-CoA + Nicotinamide adenine dinucleotide phosphate
GPR,
Lower bound,0.0
Upper bound,1000.0


In [29]:
model.metabolites.hbtcoa_c

Metabolite identifier,hbtcoa_c
Name,3-Hydroxybutyryl-CoA
Memory address,0x1ec5d7d6210
Formula,C25H42N7O18P3S
Compartment,c
In 1 reaction(s),phaB


In [30]:
# phaC reaction

new_reaction2 = Reaction('phaC', '3-Hydroxybutyryl-CoA: P3HB polymerization')  # The reaction corresponding to phaC

# phaC uses:
## 3-Hydroxybutyryl-CoA <=> P3HB + CoA
## We can find these as: (Note: All of these are in the cytoplasm)
### 3-Hydroxybutyryl-CoA = hbtcoa_c

### P3HB                 = Not Defined yet
### CoA                  = coa_c


# Since 3-Hydroxybutyryl-CoA does not exist in the model, we will have to define it before.
P3HB = Metabolite(id='P3HB_c', name='poly-3-Hydroxybutanoate', compartment='c', formula='[C4H6O2]n')


# We can now define the reaction stoichiometry:
new_reaction2.add_metabolites({
                            model.metabolites.get_by_id("hbtcoa_c"): -1,
                            P3HB: 1,
                            model.metabolites.get_by_id("coa_c"): 1
                             })

model.add_reactions([new_reaction2])

In [31]:
model.reactions.phaC

Reaction identifier,phaC
Name,3-Hydroxybutyryl-CoA: P3HB polymerization
Memory address,0x1ec5f2da6d0
Stoichiometry,hbtcoa_c --> P3HB_c + coa_c 3-Hydroxybutyryl-CoA --> poly-3-Hydroxybutanoate + Coenzyme A
GPR,
Lower bound,0.0
Upper bound,1000.0


In [32]:
model.metabolites.P3HB_c

Metabolite identifier,P3HB_c
Name,poly-3-Hydroxybutanoate
Memory address,0x1ec5f2db510
Formula,[C4H6O2]n
Compartment,c
In 1 reaction(s),phaC


In [35]:
# Here, we are testing if the intermediate compound can be produced
r_phaA_exp = model.add_boundary(model.metabolites.get_by_id('aacoa_c'), type='demand')

with model:
    model.objective = r_phaA_exp
    sol = model.optimize()
    print(sol.objective_value)

1.7894206549118252


In [33]:
# Here, we are testing if the intermediate compound can be produced
r_phaB_exp = model.add_boundary(model.metabolites.get_by_id('hbtcoa_c'), type='demand')

with model:
    model.objective = r_phaB_exp
    sol = model.optimize()
    print(sol.objective_value)

1.7621545003543546


In [34]:
# Here, we are testing if the intermediate compound can be accomulated
r_phaC_exp = model.add_boundary(model.metabolites.get_by_id('P3HB_c'), type='sink')

with model:
    model.objective = r_phaC_exp
    sol = model.optimize()
    print(sol.objective_value)

12.283130434782642


In [36]:
with model:
    model.objective = model.reactions.phaC
    P3HB_production = model.optimize().objective_value
    print("Max. p3HB production [mmol gDW^-1 h^-1]:", P3HB_production)
    print("Theoretical max. yield [mmol-DH / mmol-glucose]:", P3HB_production / (-1*model.reactions.r_1714.flux))
    max_yield_mmol=P3HB_production / (-1*model.reactions.r_1714.flux)

Max. p3HB production [mmol gDW^-1 h^-1]: 12.283130434782572


AttributeError: DictList has no attribute or entry r_1714